In [69]:
import numpy as np
import onnxruntime as ort
import onnx
import onnx_graphsurgeon as gs
from pathlib import Path
from rich import print

In [70]:
  
# path=Path("D:/Learn DL/Emil-Net/src/isolated_emil_net.onnx")

path=Path("D:/Learn DL/Emil-Net/artifacts/simplified_emil_net.onnx")



iso_onnx_model = onnx.load(path)


onnx.checker.check_model(iso_onnx_model)

In [71]:

session = ort.InferenceSession(iso_onnx_model.SerializeToString())

graph=gs.import_onnx(iso_onnx_model)


inputs = {}
for input_tensor in graph.inputs:
    shape =  input_tensor.shape
    dtype = np.float32 
    inputs[input_tensor.name] = np.random.rand(*shape).astype(dtype)


output = session.run(["trajectories","scores"], inputs)



In [72]:
def print_tensor(name:str):
    where=None
    
    for node in graph.nodes:
        if node.name == name:
            print(f"Replacing Where node: {node.name}")

            condition,X, Y = node.inputs
            print(f"condition {condition}")
            print(f"X {X}")
            print(f"Y {Y}")

            output = node.outputs[0]
            
            where = node
            
            print(f"Output: {output}")
            
    return where
        


def replace_tensor(name :str):
    
    node = print_tensor(name)
    
    ten_pow_neg6 = gs.Constant(
    name=node.inputs[1].name+ "_10pow_n4",
    values= np.full((), 1e-4, dtype=np.float32) 
)


    node.inputs[1]=ten_pow_neg6
    
    node = print_tensor(name)
    

In [73]:
print("Node 1")

replace_tensor("/encoder_blocks.0/attn/Where")


print("Node 2")

replace_tensor("/planning_decoder/decoder_blocks.0/r2r_attn/Where")

Node 1

Replacing Where node: /encoder_blocks.0/attn/Where

condition Variable (/Cast_19_output_0): (shape=[1, 9], dtype=bool)

X Constant (/encoder_blocks.0/attn/Constant_output_0): (shape=[], dtype=float32)

Y Constant (/encoder_blocks.0/attn/Constant_1_output_0): (shape=[1, 9], dtype=float32)

...

Replacing Where node: /planning_decoder/decoder_blocks.0/r2r_attn/Where

condition Variable (/planning_decoder/decoder_blocks.0/Cast_1_output_0): (shape=[6, 1], dtype=bool)

X Constant (/encoder_blocks.0/attn/Constant_output_0_10pow_n4): (shape=(), dtype=float32)

Y Constant (/planning_decoder/decoder_blocks.0/r2r_attn/Constant_1_output_0): (shape=[6, 1], dtype=float32)

Output: Variable (/planning_decoder/decoder_blocks.0/r2r_attn/Where_output_0): (shape=[6, 1], dtype=float32)

In [74]:
output_path=r"D:\Learn DL\Emil-Net\artifacts\10pow4_model.onnx"
# Save modified model
onnx.save(gs.export_onnx(graph), output_path)
print(f"Where node replaced and model saved as {output_path}")


['\x1b'][W] Found distinct tensors that share the same name:
[id: 2637598115280] Constant (/encoder_blocks.0/attn/Constant_output_0_10pow_n4): (shape=(), dtype=float32)
[id: 2637703798224] Constant (/encoder_blocks.0/attn/Constant_output_0_10pow_n4): (shape=(), dtype=float32)
Note: Producer node(s) of first tensor:
[]
Producer node(s) of second tensor:
[]


Where node replaced and model saved as D:\Learn DL\Emil-Net\artifacts\10pow4_model.onnx